In [1]:
import glob, tqdm, wandb, os, json, random, time, jax
from absl import app, flags
from ml_collections import config_flags
from log_utils import setup_wandb, get_exp_name, get_flag_dict, CsvLogger

from envs.env_utils import make_env_and_datasets
from envs.ogbench_utils import make_ogbench_env_and_datasets
from envs.robomimic_utils import is_robomimic_env

from utils.flax_utils import save_agent
from utils.datasets import Dataset, ReplayBuffer

from agents import agents
import numpy as np

/home/jellyho/miniconda3/envs/debs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/jellyho/Offline/robomimic/robomimic/scripts/setup_macros.py
)


In [1]:
from os.path import expanduser
import os

import numpy as np
import gymnasium as gym
from gymnasium.spaces import Box, Dict
import imageio
import h5py

import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
from robomimic import DATASET_REGISTRY

from utils.datasets import Dataset

/home/jellyho/miniconda3/envs/debs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/jellyho/Offline/robomimic/robomimic/scripts/setup_macros.py
)


In [7]:
def _check_dataset_exists(env_name):
    # enforce that the dataset exists
    task, dataset_type, hdf5_type = env_name.split("-")
    if hdf5_type == "image":
        visual = True
    else:
        visual = False
    if dataset_type == "mg":
        file_name = "low_dim_sparse_v15.hdf5"
    else:
        if visual:
            file_name = "image_v15.hdf5"
        else:
            file_name = "low_dim_v15.hdf5"
    download_folder = os.path.join(
        expanduser("~/.robomimic"), 
        task,
        dataset_type,
        file_name
    )
    assert os.path.exists(download_folder)
    
    return download_folder

In [8]:
env_name = 'can-mh-image'
dataset_path = _check_dataset_exists(env_name)

The history saving thread hit an unexpected error (DatabaseError('file is not a database')).History will not be written to the database.


In [10]:
env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)

In [12]:
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    render=True, 
    render_offscreen=True,
)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/jellyho/Offline/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)


Created environment with name PickPlaceCan
Action size is 7


In [19]:
def _get_max_episode_length(env_name):
    if env_name.startswith("lift"):
        return 300
    elif env_name.startswith("can"):
        return 300
    elif env_name.startswith("square"):
        return 400
    elif env_name.startswith("transport"):
        return 800
    elif env_name.startswith("tool_hang"):
        return 1000
    else:
        raise ValueError(f"Unsupported environment: {env_name}")

In [20]:
max_episode_length = _get_max_episode_length(env_name)

In [21]:
wenv = RobomimicImageWrapper(env, max_episode_length=max_episode_length)

TypeError: argument of type 'NoneType' is not iterable

In [17]:
class RobomimicImageWrapper(gym.Env):
    def __init__(self, 
        env,
        init_state=None,
        normalization_path=None,
        render_obs_keys=['agentview_image', 'robot0_eye_in_hand_image'],
        render_hw=(256, 256),
        render_camera_name="agentview",
        max_episode_length=None,
        ):

        self.env = env
        self.render_obs_keys = render_obs_keys
        self.init_state = init_state
        self.render_hw = render_hw
        self.render_camera_name = render_camera_name
        self.video_writer = None
        self.max_episode_length = max_episode_length
        self.env_step = 0
        self.n_episodes = 0

        self.seed_state_map = dict()
        self._seed = None
        self.render_cache = {}
        self.has_reset_before = False
        
        # setup spaces
        low = np.full(env.action_dimension, fill_value=-1.)
        high = np.full(env.action_dimension, fill_value=1.)
        self.action_space = Box(
            low=low,
            high=high,
            shape=low.shape,
            dtype=low.dtype,
        )

        obs_example = self.get_observation()
        observation_space = Dict()
        for key, value in obs_example.items():
            shape = value['shape']
            min_value, max_value = -1, 1
            if key.endswith('image'):
                min_value, max_value = 0, 1
            elif key.endswith('quat'):
                min_value, max_value = -1, 1
            elif key.endswith('qpos'):
                min_value, max_value = -1, 1
            elif key.endswith('pos'):
                # better range?
                min_value, max_value = -1, 1
            else:
                raise RuntimeError(f"Unsupported type {key}")
            
            this_space = Box(
                low=min_value,
                high=max_value,
                shape=shape,
                dtype=np.float32
            )
            observation_space[key] = this_space
        self.observation_space = observation_space


    def get_observation(self, raw_obs=None):
        if raw_obs is None:
            raw_obs = self.env.get_observation()
        
        for k in self.render_obs_keys:
            self.render_cache[k] = raw_obs[k]

        obs = dict()
        for key in self.observation_space.keys():
            obs[key] = raw_obs[key]
        return obs

    def seed(self, seed=None):
        np.random.seed(seed=seed)
        self._seed = seed
    
    def reset(self):
        if self.init_state is not None:
            if not self.has_reset_before:
                # the env must be fully reset at least once to ensure correct rendering
                self.env.reset()
                self.has_reset_before = True

            # always reset to the same state
            # to be compatible with gym
            raw_obs = self.env.reset_to({'states': self.init_state})
        elif self._seed is not None:
            # reset to a specific seed
            seed = self._seed
            if seed in self.seed_state_map:
                # env.reset is expensive, use cache
                raw_obs = self.env.reset_to({'states': self.seed_state_map[seed]})
            else:
                # robosuite's initializes all use numpy global random state
                np.random.seed(seed=seed)
                raw_obs = self.env.reset()
                state = self.env.get_state()['states']
                self.seed_state_map[seed] = state
            self._seed = None
        else:
            # random reset
            raw_obs = self.env.reset()

        # return obs
        obs = self.get_observation(raw_obs)
        return obs
    
    def step(self, action):
        raw_obs, reward, done, info = self.env.step(action)
        obs = self.get_observation(raw_obs)
        return obs, reward, done, info
    
    def render(self, mode='rgb_array'):
        if self.render_cache is None:
            raise RuntimeError('Must run reset or step before render.')
        img = np.moveaxis(self.render_cache, 0, -1)
        img = (img * 255).astype(np.uint8)
        return img

In [ ]:
env, eval_env, train_dataset, val_dataset = make_env_and_datasets('can-mh-image')